# 什么是Object Detection？

找到并且确定图片中的所有待监测对象

## 目标检测框的确定方法

首先基于目标检测框是一个矩形的事实，我们可以使用4个值来唯一确定一个目标检测框在图中的位置,以下是常用的两种方法，分别称为corners和midpoint

左上方和右下方顶点的坐标 
检测框中心点坐标和框的宽高


## 如何衡量预测检测框和真实检测框之间的差距？

我们一般通过 I O U ( i n t e r s e c t i o n   o v e r   u n i o n ) IOU(intersection\ over\ union) IOU(intersection over union)来衡量. A ⋂ B A ⋃ B \frac {A\bigcap B} {A\bigcup B} 
A⋃B
A⋂B
​	
A为真实框区域，B为预测框区域。
两个目标框的交集怎么求呢？
首先，我们需要意识到，两个框的交集仍然是一个框，那么就可以用左上和右下的坐标来唯一确定。
这是一张图片的坐标系，左上方是 （ 0 ， 0 ） （0，0） （0，0），右下方是 （ 1 ， 1 ） （1，1） （1,1）
假设真实框的左上角坐标是 ( x 1 , y 1 ) (x_1,y_1) (x1,y1)，预测框的左上角坐标是 ( x 2 , y 2 ) (x_2,y_2) (x2,y2),那么它们交集框的左上角坐标就是 ( m a x ( x 1 , x 2 ) , m a x ( y 1 , y 2 ) ) (max(x_1,x_2), max(y_1,y_2)) (max(x1,x2),max(y1,y2)),这里读者可以用草稿纸画一下。类似的，右下角的坐标是(min(x3,x4),min(y3,y4)) (min(x_3,x_4),min(y_3,y_4)) (min(x3,x4),min(y3,y4))

## non max suppression 非最大化抑制

当一个待探测目标有多个框检测到了它，但是我们只想保留效果最好的那一个，就需要用nms算法来去除多余的探测框。
nms中， 每个box的数据格式为【class, confidence, x1, y1, w1, h1】。其中class代表这个探测框预测了哪个类别，confidence表示这个框里包含物体的置信度，后面四个值是midpoint形式下确定一个探测框的4个点。
nms算法：
输入：boxes，包含许多box，每个box都是如上所说的格式
输出：boxes_after_nms经过非最大化抑制的boxes（清除了一些效果不好的box）
过程：

遍历boxes，将置信度低于阈值的全部删除
将剩下的boxes按照置信度从大到小排序
每次从boxes中将最大置信度的chosen_box拿出，然后遍历boxes中剩下的box，如果chosen_box和box预测的类别相同并且IOU值大于一个阈值，那么这个box就做了重复的工作，将其删除
将choice_box插入待返回的列表boxes_after_nms
重复3，4,直到boxes为空

## Mean Average Precision

MAP是评价目标探测效果好坏的一个重要指标，计算它之前需要先知道precision和recall的概念。
假设我们预测出了一个检测框，通过IOU评测方法iou以后大于了阈值，我们认为它的效果不错，就归为了正确的预测框，类似这种预测正确的样本称其为True Positive(TP)。如果IOU比阈值小，就认为这个样本是错误的，这种预测是正确的样本，但实际是错的这种，称其为False Positive（FP）。
最后假设原本正确样本（真实框）的总和是Total，我们就可以借上面提到的三个量定义precision和recall了。
 
分析每个样本预测的正确性后，我们会得到一个新的precision和recall，这样以recall为横坐标，precision为纵坐标，将每个坐标点依次连接，得出的面积就是Average Precision。
每一个class都有Average Precision， 将所有class的AP相加除以类别数就得到了MAP（mean average precision）

算法实现
输入：一个batch的预测boxes，真实框labels[ i n d e x , c l a s s , c o n f i d e n c e , x , y , w , h ] \ \ [index, class , confidence, x, y ,w, h]   [index,class,confidence,x,y,w,h],这里的index代表的是不同的图片。
输出：一个batch的mean average precision

过程：

分别计算每一个类别的mean average precision，从boxes和labels中收集对应类别的数据项，分别保存在class_boxes, class_labels中
class_boxes的长度代表detection的次数，考虑用一个tensor来记录每次detection是否为TP
对class_boxes和class_labels按照index排序再分组
遍历class_boxes时，对于每一个图片的boxes，如果对应的图片中没有真实探测框（class_labels分组后没有对应的key），该组图片直接被判为负样本
如果class_labels中有对应的key，将class_boxes中对应分组内部按照confidence从大到小排序，并去与真实框依次求IOU，得到一个最大的IOU并判断是否大于阈值，如果是则被定为正样本，否则被定为负样本。（可能预测的框有多个框与某个真实框IOU较大，只取一个当正样本，其余都为负样本）